# Extract Data

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gc 
import time

In [2]:
pd.__version__

'0.24.2'

In [3]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [4]:
%cd C:\Users\ep17\Documents\txt

C:\Users\ep17\Documents\txt


In [5]:
print('Ready to start')

Ready to start


# Helper Functions

In [6]:
def time_convert(sec):
    mins = sec // 60
    sec = sec % 60
    hours = mins // 60
    mins = mins % 60
    return ("Time Lapsed = {}:{}:{}".format(int(hours), int(mins), int(sec)))

start_time = time.time()

In [7]:
def edit_data(df, cols):
    print('Editing...')
    
    # Convert categorical variables 'missing' to NaN
    turn_to_cat = ['mba_delinquency_status', #  X = Unknown/Missing LPI date
    'orig_active_status',
    'current_investor_code', # U = No Info
    'current_product_type', # NULL = No Info
    'mba_worst_ever', # U = No Info
    'property_type', # U = No Info
    'occupancy_type', # U = No Info
    'product_type', # U = No Info
    'loan_type', # U = No Info
    'loan_purpose', # U = No Info
    'payment_frequency', # U = No Info
    'channel', # U = No Info
    'documentation_type',  # U = No Info
    'gse_eligible_flag',  # U = No Info
    'payment_frequency'] # U = No Info
    
    c_cat = []
    for i in cols:
        if i in turn_to_cat:
            c_cat.append(i)
    
    for i in c_cat:
        print(i)
        if True in df[i].astype(str).isin(['U']).values.tolist():
            df.loc[(df[i] == 'U'), i] = np.NaN
        elif True in df[i].astype(str).isin(['X']).values.tolist():
            df.loc[(df[i] == 'X'), i] = np.NaN
        elif True in df[i].astype(str).isin(['NULL']).values.tolist():
            df.loc[(df[i] == 'NULL'), i] = np.NaN
            
    # Convert binary variables 'missing' to NaN
    turn_to_dummies = ['prepay_penalty_flag',
    'collateral_type', 
    'product_type_category', 
    'loan_purpose_category', 
    'mortgage_insurance_flag', 
    'active_status', # no NA
    'bk_flag', # no NA
    'buydown_flag', 
    'convertible_flag', # no NA
    'pool_insurance_flag', 
    'negative_amortization_flag', 
    'io_flag', 
    'paid_off_flag', 
    'inferred_collateral_type']

    c_dum = []
    for i in cols:
        if i in turn_to_dummies:
            c_dum.append(i)
    
    for i in c_dum:
        print(i)
        unique_values = df[i].astype(str).unique().tolist()
        if True in df[i].astype(str).isin(['U']).values.tolist():
            unique_values.remove('U')
            df.loc[(df[i] == 'U'), i] = np.NaN
        
        if len(unique_values) > 2:
            print("ERROR!!!!!! more than 2 unique values")
            print(unique_values, '| Removing...', unique_values[2])
            df.loc[(df[i] == unique_values[2]).values, i] = np.NaN        
    
    print('Done editing. {} \n'.format(time_convert(time.time() - start_time)))
    return df

In [8]:
def add_loan_counts(df):
    df['one'] = 1
    counts_df = df.groupby(by=['loan_id'])[['one']].sum().reset_index()
    counts_df.columns = ['loan_id', 'counts']
    df = df.merge(counts_df, on='loan_id', how='inner', validate='many_to_one')
    df = df.drop(columns=['one'])
    print('Done merging \n')
    return df

In [9]:
# Time code execution
def time_convert(sec):
    mins = sec // 60
    sec = sec % 60
    hours = mins // 60
    mins = mins % 60
    return ("Time Lapsed = {}:{}:{}".format(int(hours), int(mins), int(sec)))
    
start_time = time.time()

In [10]:
# Get range of values of each column in specified dataframe
def get_ranges(df):
    neg_numbers = []
    df = df.select_dtypes(include=np.number)
    for col in df.columns:
        print('{} range of values: [{}, {}]'.format(col, df[col].min(), df[col].max()))
        if df[col].min() < 0:
            neg_numbers.append(col)
    return neg_numbers

In [11]:
# Remove rows with strange values in any column
def remove_strange(df, neg_cols):
    for col in neg_cols:
        print(col)
        display(df[df[col] < 0])
        df = df[df[col] > 0]

    display(df)
    return df

In [12]:
# Get name of dataframe (for printing purposes)
def get_df_name(df):
    if df is all_performances:
        name = 'Performances Dataset'
    elif df is all_origination:
        name = 'Origination Dataset'
    else:
        name = ''
    return name

In [13]:
# Count the number of null values in each column and each row
def count_null(df, thresh):
    print('***{}***'.format(get_df_name(df)))
    print('Columns = {}. Rows = {}.'.format(df.shape[1], df.shape[0]))
    t = str(int(thresh*100))

    null_rows = df.isnull().sum(axis=1).values
    fiftyp_rows = null_rows/len(df.columns)
    fiftyp_rows = len(fiftyp_rows[fiftyp_rows>thresh])
    print('There are no rows with more than {} NaN columns. {} rows have more than {}% NaN columns.'.format(null_rows.max(), fiftyp_rows, t))

    null_cols = df.isnull().sum(axis=0).values
    fiftyp_cols = null_cols/len(df)
    fiftyp_cols = len(fiftyp_cols[fiftyp_cols>thresh])
    print('There are no columns with more than {} NaN rows. {} columns have more than {}% NaN rows. \n'.format(null_cols.max(), fiftyp_cols, t))

    return

In [14]:
# Remove columns with > x NaN values in the rows
def drop_columns(df, thresh):
    print('***{}***'.format(get_df_name(df)))
    print('Shape before: {}'.format(df.shape))
    cutoff = len(df) * thresh
    df = df.drop(columns = (df.columns[df.isna().sum().values > cutoff]))
    print('Shape after: {} \n'.format(df.shape))
    return df

# Performance Data

In [15]:
def extract_performance_data(year):
    
    cols = ['loan_id', 'current_balance', 'current_interest_rate', 
             'total_payment_due', 'scheduled_principal', 'scheduled_monthly_pi', 
             'mba_delinquency_status', 'mba_days_delinquent', 'active_status', 
             'period_of_payment', 'current_investor_code', 'current_product_type', 
             'loan_age', 'mba_worst_ever', 'bk_flag']
    
    i=1
    for chunk in pd.read_csv(('Performance_Firsts_' + year + '.txt'), sep='|', usecols = cols, low_memory=False, chunksize=50000000):
        performance_data = chunk if i == 1 else pd.concat([performance_data, chunk])
        i += 1

    performance_data['year'] = int(year)
    
    print('Extracting', year, 'performance data: done')
    
    performance_data = edit_data(performance_data, cols)
    
    return performance_data

In [16]:
performance_2019 = extract_performance_data('2019')
performance_2018 = extract_performance_data('2018')
all_performances = performance_2019.append(performance_2018)
del performance_2019
del performance_2018
gc.collect()

print('\nAppending 18-19 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2017 = extract_performance_data('2017')
all_performances = all_performances.append(performance_2017)
del performance_2017
gc.collect()

print('\nAppending 17 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2016 = extract_performance_data('2016')
all_performances = all_performances.append(performance_2016)
del performance_2016
gc.collect()

print('\nAppending 16 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2015 = extract_performance_data('2015')
all_performances = all_performances.append(performance_2015)
del performance_2015
gc.collect()

print('\nAppending 15 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2014 = extract_performance_data('2014')
all_performances = all_performances.append(performance_2014)
del performance_2014
gc.collect()

print('\nAppending 14 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2013 = extract_performance_data('2013')
all_performances = all_performances.append(performance_2013)
del performance_2013
gc.collect()

print('\nAppending 13 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2012 = extract_performance_data('2012')
all_performances = all_performances.append(performance_2012)
del performance_2012
gc.collect()

print('\nAppending 12 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2011 = extract_performance_data('2011')
all_performances = all_performances.append(performance_2011)
del performance_2011
gc.collect()

print('\nAppending 11 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2010 = extract_performance_data('2010')
all_performances = all_performances.append(performance_2010)
del performance_2010
gc.collect()

print('\nAppending 10 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2009 = extract_performance_data('2009')
all_performances = all_performances.append(performance_2009)
del performance_2009
gc.collect()

print('\nAppending 09 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

performance_2008 = extract_performance_data('2008')
all_performances = all_performances.append(performance_2008)
del performance_2008
gc.collect()

print('\nAppending 08 performance data: done \n')
print(time_convert(time.time() - start_time), '\n')

print('\nAppending performance data: done')

Extracting 2019 performance data: done
Editing...
mba_delinquency_status
current_investor_code
current_product_type
mba_worst_ever
active_status
bk_flag
Done editing. Time Lapsed = 0:25:47 

Extracting 2018 performance data: done
Editing...
mba_delinquency_status
current_investor_code
current_product_type
mba_worst_ever
active_status
bk_flag
Done editing. Time Lapsed = 0:55:51 


Appending 18-19 performance data: done 

Time Lapsed = 0:57:38 

Extracting 2017 performance data: done
Editing...
mba_delinquency_status
current_investor_code
current_product_type
mba_worst_ever
active_status
bk_flag
Done editing. Time Lapsed = 1:30:24 


Appending 17 performance data: done 

Time Lapsed = 1:34:11 

Extracting 2016 performance data: done
Editing...
mba_delinquency_status
current_investor_code
current_product_type
mba_worst_ever
active_status
bk_flag
Done editing. Time Lapsed = 2:8:24 


Appending 16 performance data: done 

Time Lapsed = 2:14:19 

Extracting 2015 performance data: done
Editin

In [17]:
all_performances.to_csv('PERFORMANCES.csv')
print(time_convert(time.time() - start_time))

Time Lapsed = 25:7:23


# Origination Data

In [18]:
def extract_origination_data(filename):
    
    print('Extracting {}'.format(filename))
    
    cols = ['loan_id', 'origination_date', 'property_zip', 'state', 'property_type', 
            'number_of_units', 'occupancy_type', 'original_balance', 
            'sale_price', 'appraised_value', 'product_type', 'original_term',
            'initial_interest_rate', 'back_end_ratio', 'loan_type', 
            'loan_purpose', 'payment_frequency', 'channel', 'buydown_flag', 
            'documentation_type',  'convertible_flag', 'pool_insurance_flag', 'original_ltv', 
            'negative_amortization_flag', 'margin', 'periodic_rate_cap', 
            'periodic_rate_floor', 'lifetime_rate_cap', 'lifetime_rate_floor',
            'rate_reset_frequency', 'pay_reset_frequency', 'first_rate_reset_period', 
            'fico_score_at_origination', 'prepay_penalty_flag', 'prepay_penalty_term', 
            'combined_ltv_at_origination', 'cbsa', 'io_term', 'io_flag', 
            'msa', 'paid_off_flag', 'inferred_collateral_type', 'collateral_type', 
            'orig_active_status', 'period', 'product_type_category', 'loan_purpose_category', 
            'mortgage_insurance_flag', 'gse_eligible_flag', 'payment_frequency']
    
    origination_data = pd.read_csv(filename, sep='|', low_memory=False, usecols = cols)
        
    origination_data = edit_data(origination_data, cols)
        
    return origination_data

In [19]:
inactive_origination_2008_2010 = extract_origination_data('Inactive_Origination_Firsts_2008_2010.txt')

active_origination_202007 = extract_origination_data('Recent_Origination_Firsts_202007.txt')

all_origination = inactive_origination_2008_2010.append(active_origination_202007)
print('\nAppending origination data: done')
print(time_convert(time.time() - start_time))

del inactive_origination_2008_2010
del active_origination_202007
gc.collect()

Extracting Inactive_Origination_Firsts_2008_2010.txt
Editing...
property_type
occupancy_type
product_type
loan_type
loan_purpose
payment_frequency
channel
documentation_type
orig_active_status
gse_eligible_flag
payment_frequency
buydown_flag
convertible_flag
pool_insurance_flag
negative_amortization_flag
prepay_penalty_flag
io_flag
paid_off_flag
inferred_collateral_type
collateral_type
product_type_category
loan_purpose_category
mortgage_insurance_flag
Done editing. Time Lapsed = 25:9:5 

Extracting Recent_Origination_Firsts_202007.txt
Editing...
property_type
occupancy_type
product_type
loan_type
loan_purpose
payment_frequency
channel
documentation_type
orig_active_status
gse_eligible_flag
payment_frequency
buydown_flag
convertible_flag
pool_insurance_flag
negative_amortization_flag
prepay_penalty_flag
io_flag
paid_off_flag
inferred_collateral_type
collateral_type
product_type_category
loan_purpose_category
mortgage_insurance_flag
Done editing. Time Lapsed = 25:21:47 


Appending orig

21

In [20]:
all_origination.to_csv('ORIGINATION.csv')
print(time_convert(time.time() - start_time))

Time Lapsed = 25:48:24


# Save as Chunks

In [21]:
# neg_cols = get_ranges(performances)
# print(neg_cols)
# # performances = remove_strange(performances, neg_cols)

In [27]:
all_performances = drop_columns(all_performances, 0.75)
all_origination = drop_columns(all_origination, 0.75)

***Performances Dataset***
Shape before: (4995041139, 16)
Shape after: (4995041139, 16) 

***Origination Dataset***
Shape before: (72399685, 49)
Shape after: (72399685, 38) 



In [ ]:
i=1
for chunk in pd.read_csv('PERFORMANCES.csv', low_memory=False, chunksize=20000000):
    all_performances = chunk if i == 1 else pd.concat([all_performances, chunk])
    i += 1

In [ ]:
all_origination = pd.read_csv('ORIGINATION.csv', low_memory=False)

In [ ]:
merged_df = all_performances.merge(all_origination, on='loan_id', how='inner', validate='many_to_one')
print(time_convert(time.time() - start_time))

In [ ]:
merged_df.to_csv('MERGED_orig2.csv')
print(time_convert(time.time() - start_time))

In [ ]:
merged_df['origination_date'] = merged_df['origination_date'].astype(int)
unique_dates = merged_df['origination_date'].unique()
unique_dates = unique_dates[unique_dates > 200800]

for date in unique_dates:
    for state in merged_df['state'].unique():
        file_name = 'MERGED_{}_{}.csv'.format(date, state)
        condition = (merged_df['origination_date'] == date) & (merged_df['state'] == state)
        merged_df[condition].to_csv(file_name)

In [ ]:
print('DONE')